In [1]:
import numpy as np
from data import Vertebral_column
from data import Co_Author
from Processing_Data import Abanole
from Processing_Data import Ecoli, Ecoli_All
from Processing_Data import Ecloli1
from Processing_Data import Ecoli3
from Processing_Data import Glass1
from Processing_Data import Glass4
from Processing_Data import Haberman
from Processing_Data import Waveform
from Processing_Data import New_thyroid2
from Processing_Data import Page_blocks
from Processing_Data import Pima_Indians_Diabetes, Pima
from Processing_Data import Satimage
from Processing_Data import Transfusion
from Processing_Data import Yeast
from Processing_Data import Haberman_All
from Processing_Data import Transfusion_All
from Processing_Data import PimaIndians_All
from data import indian_liver_patient
#from data import spect_heart
from wsvm.application import Wsvm
from svm.application import Svm
from sklearn.svm import SVC
#from sklearn.metrics import f1_score
from sklearn.metrics  import classification_report,precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import _safe_indexing
from sklearn import metrics
import math
from datetime import datetime
from fuzzy.weight import fuzzy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from Processing_Data import Ecoli_Kfold
from Processing_Data import Haberman_KFold
from Processing_Data import Transfution_Kfold
from Processing_Data import Co_Author
from Processing_Data import Co_Author_TestSize
import csv
from data import Vertebral_column

(array([[-0.20356986, -0.23616822, -0.01535438, ..., -0.10297836,
        -0.00333545,  1.9115189 ],
       [-0.71319985, -0.07417361,  0.33070229, ..., -0.10297836,
        -1.03616576, -0.16111872],
       [-0.42198271,  0.16881831, -1.05352439, ..., -0.10297836,
         0.16880293, -0.53796193],
       ...,
       [ 0.59727728,  0.08782101,  0.67675896, ..., -0.10297836,
        -2.75754961, -0.53796193],
       [-0.13076557, -0.39816283,  1.48422452, ..., -0.10297836,
         1.37377162, -0.53796193],
       [ 0.014843  ,  0.08782101,  0.79211118, ..., -0.10297836,
        -0.34761222, -0.53796193]]), array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,

In [2]:
def svm_lib(X_train, y_train,X_test):
    svc=SVC(probability=True, kernel='linear')
    model = svc.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [3]:
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [4]:
def svm(C,X_train, y_train,X_test):
    model = Svm(C)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [5]:
def is_tomek(X,y, class_type):
    print(y)
    print(type(y))
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1]
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample:
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample)
                X_dangxet.append(nn_index[index_sample])
                links[index_sample] = True
                
    return links,X_dangxet,X_tl

In [6]:
def is_tomek_new(X,y, class_type):
    # print(y)
    # print(type(y))
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1]
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample:
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample)
                X_dangxet.append(nn_index[index_sample])
                links[index_sample] = True

    stt = np.zeros(len(X_dangxet), dtype=int)
    arr_tlp = np.stack((X_dangxet, X_tl, stt), axis=1)

    return arr_tlp

In [7]:
X, y = Ecoli_Kfold.load_data()
arr = is_tomek_new(X, y, class_type=[-1.0])
print(arr)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1

In [8]:
X, y = Ecoli_Kfold.load_data()
X = np.array(X)
links, positive_ind, negative_ind = is_tomek(X, y, class_type=[-1.0])
# links.tolist()
print(positive_ind)
print(negative_ind)
pos_index = np.where(y == 1)[0]
neg_index = np.where(y == -1)[0]
print(X[positive_ind])
for ind,i in enumerate(positive_ind): 
    # print(ind, i)
    print(X[i])
# stt = np.zeros(len(positive_ind), dtype=int)
# con = np.stack((positive_ind, negative_ind,stt), axis=1)
# print(con)
# print(con[0][2])

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1

In [9]:
def Gmean(y_test,y_pred):
    cm_WSVM = metrics.confusion_matrix(y_test, y_pred)
    sensitivity = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    specificity = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(sensitivity*specificity)
    return specificity,sensitivity,gmean

In [10]:
def metr(X_train,y_test,test_pred,se,sp,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    print("So luong samples: ",len(X_train))
    print("\n",classification_report(y_test, test_pred))
    print("SP      : ",sp)
    print("SE      : ",se)
    print("Gmean   : ",gmean)
    print("F1 Score: ",f1_score(y_test, test_pred))
    print("Accuracy: ",accuracy_score(y_test,test_pred))
    print("AUC     : ",roc_auc_score(y_test, test_pred))
    print("Ma tran nham lan: \n",confusion_matrix(y_test, test_pred))
    

In [11]:
def metr_text(f,X_train,y_test,test_pred,sp,se,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    f.write(f"\n\nSo luong samples Tong: {len(X_train)+len(y_test)}")
    f.write(f"\n\nSo luong samples training: {len(X_train)}")
    f.write(f"\nSo luong samples testing: {len(y_test)}\n")
    f.write("\n"+str(classification_report(y_test, test_pred)))
    f.write(f"\nSP      : {sp:0.4f}")
    f.write(f"\nSE      : {se:0.4f}")
    f.write(f"\nGmean   : {gmean:0.4f}")
    f.write(f"\nF1 Score: {f1_score(y_test, test_pred):0.4f}")
    f.write(f"\nAccuracy: {accuracy_score(y_test,test_pred):0.4f}")
    f.write(f"\nAUC     : {roc_auc_score(y_test, test_pred):0.4f}")
    f.write("\n\nMa tran nham lan: \n"+str(confusion_matrix(y_test, test_pred)))

In [12]:
def compute_weight(X, y,name_method ="actual_hyper_lin", name_function = "exp", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            d = method.own_class_center_divided(X, y)
        elif name_method == "distance_center_own_opposite_tam":
            d_own, d_opp, d_tam = method.distance_center_own_opposite_tam(X,y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
        elif name_function == "func_own_opp_new":
            W = function.func_own_opp_new(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v1":
            W = function.func_own_opp_new_v1(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v2":
            W = function.func_own_opp_new_v2(d_own,d_opp,pos_index,neg_index,d_tam)
    except Exception as e:
        print('dont exist function')
        print(e)
    # pos_index = np.where(y == 1)[0]
    # neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [13]:
import copy
from Processing_Data import Test
X, y = Test.load_data()
X = np.array(X)
y = np.array(y)
pos_ind = np.where(y == 1)[0]
neg_ind = np.where(y == -1)[0]
def distance_center_own_opposite_tam(X,y):
    X_own = copy.deepcopy(X)
    X_opposite = copy.deepcopy(X)
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    x_cenpos = np.mean(X[pos_index], axis=0)
    x_cenneg = np.mean(X[neg_index], axis=0)

    # compute distance
    X_own[pos_index] = X_own[pos_index] - x_cenpos
    X_own[neg_index] = X_own[neg_index] - x_cenneg

    X_opposite[pos_index] = X_opposite[pos_index] - x_cenneg
    X_opposite[neg_index] = X_opposite[neg_index] - x_cenpos

    x_2tam = x_cenneg - x_cenpos

    d_cen_own = np.linalg.norm(X_own, axis=1)
    d_cen_opposite = np.linalg.norm(X_opposite, axis=1)
    d_tam = np.linalg.norm(x_2tam, axis=0)
    return d_cen_own, d_cen_opposite, d_tam
distance_center_own_opposite_tam(X,y)

d_cenpos, d_cenneg, d_tam = distance_center_own_opposite_tam(X,y)
dmax_pos = np.max(d_cenpos[pos_ind])
dmin_pos = np.min(d_cenpos[neg_ind])
dmax_neg = np.max(d_cenneg[neg_ind])
dmin_neg = np.min(d_cenneg[pos_ind])

print(d_cenpos)
print(dmax_pos)
print(dmin_pos)

print(d_cenneg)
print(dmax_neg)
print(dmin_neg)

def func_own_opp_new_v1(d_cenpos, d_cenneg, pos_ind, neg_ind, d_tam, delta=1e-6):
    f = np.zeros(len(d_cenpos))
    dmax_pos = np.max(d_cenpos[pos_ind])
    dmin_pos = np.min(d_cenpos[neg_ind])
    dmax_neg = np.max(d_cenneg[neg_ind])
    dmin_neg = np.min(d_cenneg[pos_ind])
    f[pos_ind] = 1 - (d_cenpos[pos_ind] + dmin_neg/d_cenneg[pos_ind])/(dmax_pos+dmin_neg+d_tam+delta)
    f[neg_ind] = 1 - (d_cenneg[neg_ind] + dmin_pos/d_cenpos[neg_ind])/(dmax_neg+dmin_pos+d_tam+delta)
    return f[pos_ind], f[neg_ind], f
func_own_opp_new_v1(d_cenpos, d_cenneg, pos_ind, neg_ind, d_tam, delta=1e-6)

[1.1088733  1.13578167 0.50990195 0.62417946 0.97447422 0.8219489
 0.67082039 0.88746831]
1.1357816691600546
0.6241794613730894
[3.35410197 2.36845941 3.43214219 2.95803989 2.6925824  3.09515751
 3.68640747 3.93700394]
3.9370039370059056
2.368459414893994


(array([0.67699534, 0.81852059, 0.80138252]),
 array([0.48925437, 0.48390272, 0.56538842, 0.49739736, 0.39493747]),
 array([0.48925437, 0.67699534, 0.81852059, 0.48390272, 0.56538842,
        0.49739736, 0.80138252, 0.39493747]))

In [14]:
def fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction):
    if namemethod =="own_class_center_opposite" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_center)
    elif namemethod =="own_class_center" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="own_class_center_divided" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="estimated_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="actual_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_actual)
    else:   
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction)
    return distribution_weight

In [15]:
y = np.array([5,6,7,8,9,10])
a = np.where(y>5)
a

(array([1, 2, 3, 4, 5], dtype=int64),)

In [16]:
def data_tomelinks(f,C,weight,X_test,y_test,X_train,y_train,n_neighbors,clf=None,namemethod=None,namefunction=None):
    links,ind_posX,ind_negX = is_tomek(X_train,y_train,class_type=[-1.0])
    #print(len(ind_posX))
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,new_W)
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    specificity,sensitivity,gmean = Gmean(y_test,y_predict)
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)
    y_nn = [] #chứa nhãn của k mẫu dữ liệu gần nhất với mẫu được xét

    # Mẫu âm bị phân loại sai -> Giảm trọng số của mẫu âm đó
    neg_pred = clf.predict(X_train[neg_index])
    idx_neg_wrong = np.where(neg_pred != -1.0)
    new_W[idx_neg_wrong] =  new_W[idx_neg_wrong]*0.5 # giam manh (Co-Author*0.9)

    # Tăng, giảm trọng số của các mẫu trong TLPs
    ind_nn = [] # chứa chỉ số của các mẫu dương bị phân loại sai trong ind_posX
    for ind,i in enumerate(ind_posX): 
        y_pred = clf.predict([X_train[i]])  #
        if y_pred == -1.0: # dương bị phân loại sai
            ind_nn.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[i]])[1]).tolist()  
            for j in knn_X[0]:
                y_nn.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn
        else: # dương được phân loại đúng
            new_W[ind_negX[ind]] = new_W[ind_negX[ind]]/1.2 #Co-Author/1.05
            new_W[i] = new_W[i]*1.2 #Co-Author*1.05
    ind_nn = np.array(ind_nn)
    #print(ind_nn)
    y_nn = np.array(y_nn)
    if len(y_nn)>0:
        y_nn = np.array_split(y_nn, len(y_nn)/n_neighbors) 
    #print(len(y_nn))
    for ind,i in enumerate(range(0,len(y_nn))):   #
        if 1 not in y_nn[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu dương -> giảm trọng số
            new_W[ind_posX[ind_nn[ind]]] = new_W[ind_posX[ind_nn[ind]]]*0.5
            #print(ind_posX[ind_nn[ind]])
        else:
            # print("Old Neg: ",new_W[ind_negX[ind_nn[ind]]])
            # print("Old Pos: ",new_W[ind_posX[ind_nn[ind]]])
            new_W[ind_negX[ind_nn[ind]]] = new_W[ind_negX[ind_nn[ind]]]/1.2
            new_W[ind_posX[ind_nn[ind]]] = new_W[ind_posX[ind_nn[ind]]]*1.2
            # print("New Neg: ",new_W[ind_negX[ind_nn[ind]]])
            # print("New Pos: ",new_W[ind_posX[ind_nn[ind]]])

    return new_W,gmean,sensitivity

In [17]:
from sklearn.neighbors import KNeighborsClassifier
X, y = Ecoli_Kfold.load_data()
X = np.array(X)
y_nn = []
ind_nn = []
links, positive_ind, negative_ind = is_tomek(X, y, class_type=[-1.0])
# links.tolist()
print(positive_ind)
print(negative_ind)
pos_index = np.where(y == 1)[0]
neg_index = np.where(y == -1)[0]
nn2 = NearestNeighbors(n_neighbors=5)
nn2.fit(X)
clf = KNeighborsClassifier(n_neighbors= 9)
clf.fit(X,y)
for ind,i in enumerate(positive_ind): 
    y_pred = clf.predict([X[i]]) 
    if y_pred == -1.0: # dương bị phân loại sai
        ind_nn.append(ind)
    # print(negative_ind[ind])
        knn_X = (nn2.kneighbors([X[i]])[1]).tolist()  
        # print(knn_X)
        for j in knn_X[0]:
            y_nn.append(y[j])
print(len(y_nn))
print(y_nn)
if len(y_nn)>0:
    y_nn = np.array_split(y_nn, len(y_nn)/5) 
print(y_nn)

print(range(0,len(y_nn)))
for ind,i in enumerate(range(0,len(y_nn))):
    if 1 not in y_nn[i][1:]:
        print(y_nn[i][1:])
        print(positive_ind[ind_nn[ind]])


[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1

In [18]:
def data_tomelinks_new(f,C,weight,X_test,y_test,X_train,y_train,n_neighbors,clf=None,namemethod=None,namefunction=None):
    links,ind_posX,ind_negX = is_tomek(X_train,y_train,class_type=[-1.0])
    #print(len(ind_posX))
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,new_W)
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    specificity,sensitivity,gmean = Gmean(y_test,y_predict)
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)

    # Mẫu âm bị phân loại sai -> Giảm trọng số của mẫu âm đó
    neg_pred = clf.predict(X_train[neg_index])
    idx_neg_wrong = np.where(neg_pred != -1.0)
    new_W[idx_neg_wrong] =  new_W[idx_neg_wrong]*0.5 # giam manh (Co-Author*0.9)

    # Tăng, giảm trọng số của các mẫu trong TLPs
    # Trường hợp 1, 2, 3, 4
    ind_nn_pos = [] # chứa chỉ số của các mẫu dương bị phân loại sai trong ind_posX
    y_nn_pos = [] #chứa nhãn của k mẫu dữ liệu gần nhất với mẫu dương được xét
    ind_nn_neg = [] # chứa chỉ số của các mẫu âm bị phân loại sai trong ind_negX
    y_nn_neg = [] # chứa nhãn của k mẫu dữ liệu gần nhất với mẫu âm được xét

    for ind,i in enumerate(ind_posX): 
        y_pred = clf.predict([X_train[i]])
        if y_pred == -1.0: # dương bị phân loại sai (3,4)
            ind_nn_pos.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[i]])[1]).tolist()  
            for j in knn_X[0]:
                y_nn_pos.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn
        else: # dương được phân loại đúng (1,2)
            new_W[ind_negX[ind]] = new_W[ind_negX[ind]]/1.2 #Co-Author/1.05
            new_W[i] = new_W[i]*1.2 #Co-Author*1.05
            
    ind_nn_pos = np.array(ind_nn_pos)
    y_nn_pos = np.array(y_nn_pos)
    if len(y_nn_pos)>0:
        y_nn_pos = np.array_split(y_nn_pos, len(y_nn_pos)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_pos))):   #
            if 1 not in y_nn_pos[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu dương -> giảm mạnh trọng số
                new_W[ind_posX[ind_nn_pos[ind]]] = new_W[ind_posX[ind_nn_pos[ind]]]*0.5
            else:
                new_W[ind_negX[ind_nn_pos[ind]]] = new_W[ind_negX[ind_nn_pos[ind]]]/1.2
                new_W[ind_posX[ind_nn_pos[ind]]] = new_W[ind_posX[ind_nn_pos[ind]]]*1.2

    # Trường hợp 2

    for ind,i in enumerate(ind_negX): 
        y_pred_neg = clf.predict([X_train[i]])  #
        if y_pred_neg == 1.0: # âm bị phân loại sai
            ind_nn_neg.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[i]])[1]).tolist()  
            for j in knn_X[0]:
                y_nn_neg.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn_neg

    ind_nn_neg = np.array(ind_nn_neg)
    y_nn_neg = np.array(y_nn_neg)
    if len(y_nn_neg)>0:
        y_nn_neg = np.array_split(y_nn_neg, len(y_nn_neg)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_neg))):   #
            if -1 not in y_nn_neg[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu âm -> giảm mạnh trọng số
                new_W[ind_negX[ind_nn_neg[ind]]] = new_W[ind_negX[ind_nn_neg[ind]]]*0.5
            else:
                new_W[ind_negX[ind_nn_neg[ind]]] = new_W[ind_negX[ind_nn_neg[ind]]]/1.2
                new_W[ind_posX[ind_nn_neg[ind]]] = new_W[ind_posX[ind_nn_neg[ind]]]*1.2


    return new_W,gmean,sensitivity

In [19]:
def data_tomelinks_new1(C,weight,X_test,y_test,X_train,y_train,n_neighbors,arr_tlp,clf=None,namemethod=None,namefunction=None):
    ro1 = 0.1
    ro3 = 0.1
    ro4 = 0.5
    ro2 = 0.5
    # links,ind_posX,ind_negX = is_tomek(X_train,y_train,class_type=[-1.0])
    
    #print(len(ind_posX))
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,new_W)
    clf.fit(X_train, y_train)    
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)

    # Mẫu âm bị phân loại sai -> Giảm trọng số của mẫu âm đó
    neg_pred = clf.predict(X_train[neg_index])
    idx_neg_wrong = np.where(neg_pred != -1.0)
    new_W[idx_neg_wrong] =  new_W[idx_neg_wrong]*0.5 # giam manh (Co-Author*0.9)

    # Tăng, giảm trọng số của các mẫu trong TLPs
    # Trường hợp 1, 2, 3, 4
    ind_nn_pos = [] # chứa chỉ số của các mẫu dương bị phân loại sai trong ind_posX
    y_nn_pos = [] #chứa nhãn của k mẫu dữ liệu gần nhất với mẫu dương được xét
    ind_nn_neg = [] # chứa chỉ số của các mẫu âm bị phân loại sai trong ind_negX
    y_nn_neg = [] # chứa nhãn của k mẫu dữ liệu gần nhất với mẫu âm được xét

    for ind,i in enumerate(arr_tlp):
        y_pred_pos = clf.predict([X_train[arr_tlp[ind][0]]]) #positive
        y_pred_neg = clf.predict([X_train[arr_tlp[ind][1]]]) #negative
        if (y_pred_pos == 1) and (y_pred_neg == 1): #dương dự đoán đúng, âm dự đoán sai
            new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro1) # tăng trọng số mẫu dương
            new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]*(1 - ro1) # giảm trọng số mẫu âm
            arr_tlp[ind][2] = 1

            ind_nn_neg.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[arr_tlp[ind][1]]])[1]).tolist()  #Xem lại
            for j in knn_X[0]:
                y_nn_neg.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn_neg

        if (y_pred_pos == -1) and (y_pred_neg == -1): #âm dự đoán đúng, dương dự đoán sai
            new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro3) 
            new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]*(1 - ro3)
            arr_tlp[ind][2] = 3

            ind_nn_pos.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[arr_tlp[ind][0]]])[1]).tolist() #Xem lại 
            for j in knn_X[0]:
                y_nn_pos.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn

    ind_nn_neg = np.array(ind_nn_neg)
    y_nn_neg = np.array(y_nn_neg)
    if len(y_nn_neg)>0:
        y_nn_neg = np.array_split(y_nn_neg, len(y_nn_neg)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_neg))):   #
            if -1 not in y_nn_neg[i][1:]:      # Nếu không tồn tại nhãn -1 xung quanh X_train[i] bị dự đoán sai => nhiễu âm -> giảm mạnh trọng số
                for a in arr_tlp[[ind_nn_neg[ind]]]:
                    new_W[a[1]] = new_W[a[1]]*ro2
                arr_tlp[ind][2] = 2

    ind_nn_pos = np.array(ind_nn_pos)
    y_nn_pos = np.array(y_nn_pos)
    if len(y_nn_pos)>0:
        y_nn_pos = np.array_split(y_nn_pos, len(y_nn_pos)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_pos))):   #
            if 1 not in y_nn_pos[i][1:]:      # Nếu không tồn tại nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu dương -> giảm mạnh trọng số
                for a in arr_tlp[[ind_nn_pos[ind]]]:
                    new_W[a[0]] = new_W[a[0]]*ro4
                arr_tlp[ind][2] = 4

    return new_W

In [20]:
from sklearn.neighbors import KNeighborsClassifier
X, y = Ecoli_Kfold.load_data()
X = np.array(X)
y_nn = []
ind_nn = []
arr_tlp = is_tomek_new(X, y, class_type=[-1.0])
links, positive_ind, negative_ind = is_tomek(X, y, class_type=[-1.0])

pos_index = np.where(y == 1)[0]
neg_index = np.where(y == -1)[0]
nn2 = NearestNeighbors(n_neighbors=5)
nn2.fit(X)
clf = KNeighborsClassifier(n_neighbors= 9)
clf.fit(X,y)
print(arr_tlp)
ind_nn_neg = []
y_nn_neg = []
for ind,i in enumerate(arr_tlp):
    y_pred_pos = clf.predict([X[arr_tlp[ind][0]]]) #positive
    y_pred_neg = clf.predict([X[arr_tlp[ind][1]]]) #negative
    # print(y_pred_pos)
    # print(y_pred_neg)
    # print("***")
    if (y_pred_pos == 1) and (y_pred_neg == 1): #dương dự đoán đúng
        # new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro1) 
        # new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]/(1 - ro1)
        # arr_tlp[ind][2] = 1
        ind_nn_neg.append(ind)                          
        knn_X = (nn2.kneighbors([X[arr_tlp[ind][1]]])[1]).tolist()  
        for j in knn_X[0]:
            y_nn_neg.append(y[j])
            # print(y_nn_neg)
# print(arr_tlp)

ind_nn_neg = np.array(ind_nn_neg)
y_nn_neg = np.array(y_nn_neg)
print(y_nn_neg)
print(ind_nn_neg)
if len(y_nn_neg)>0:
    y_nn_neg = np.array_split(y_nn_neg, len(y_nn_neg)/5) 
    for ind,i in enumerate(range(0,len(y_nn_neg))):   #
        if -1 not in y_nn_neg[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu âm -> giảm mạnh trọng số
            # new_W[arr_tlp[ind][1][[ind_nn_neg[ind]]]] = new_W[ind_negX[ind_nn_neg[ind]]]/ro2
            # arr_tlp[ind][2] = 2
            # print("element:",arr_tlp[[ind_nn_neg[ind]]])
            for a in arr_tlp[[ind_nn_neg[ind]]]:
               print(X[a[1]])
            # print([ind_nn_neg[ind]])
            # new_W[ind_posX[ind_nn[ind]]]
            
            
                
# print(arr_tlp)
# for ind,i in enumerate(negative_ind): 
#     y_pred = clf.predict([X[i]]) 
#     print(y_pred)


[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1

In [21]:
x = [ [0, 1], [2, 3], [4,5]]
for ind, i in enumerate(x):
    # for j in range(len(x[i])):
        print(ind, x[ind][1])

0 1
1 3
2 5


In [22]:
def lfb(f,C,weight,namemethod,namefunction,T,X_test,y_test,X_train,y_train,n_neighbors,thamso1,thamso2): #loop find the best result
    gmax = 0
    tmax = 0
    semax = 0
    t_semax = 0
    for i in range(0,1):
        f.write(f"\t\t Vong Lap thu: T = {i+1}")
        f.write(f"\n===================================================================================================================")
        f.write(f"\n\n\tFuzzy SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
        weight, gmeanfirst, SEfirst = data_tomelinks(f,C,weight,X_test,y_test,X_train,y_train,n_neighbors,clf=None,namemethod=namemethod,namefunction=namefunction)
        #distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
        clf = Wsvm(C,weight)
        #print(X_train)
        #print(y_train)
        clf.fit(X_train, y_train)
        #print(clf.predict([X_test_val[5]]))
        pred2 = clf.predict(X_test)    #X_val
        sp,se,gmean = Gmean(y_test,pred2) #y_val
        pred_all = clf.predict(X_test)    #X_val
        sp_all,se_all,gmean_all = Gmean(y_test,pred_all)

        if(gmeanfirst > gmax):
            gmax = gmeanfirst
            tmax = i+100
        if (gmean_all > gmax):
            gmax = gmean_all
            tmax = i
        if (SEfirst > semax):
            semax = SEfirst
            t_semax = i+100
        if (se_all > semax):
            semax = se_all
            t_semax = i   
        #metr(X_train,y_test_val,pred2,sp,se,gmean)
        f.write(f"\n\n\t****** Danh gia tren tap Test:\n")
        metr_text(f,X_train,y_test,pred_all,sp_all,se_all,gmean_all)
        # if ((gmeanfirst - gmean) <= thamso1) or (gmeanfirst > thamso2):
        #     f.write("\n_____Gmean_ERROR!!!____\n")
        #     print("\n_____Gmean_ERROR!!!____\n")
        #     return X_train, y_train
        # else:
        #     gmean = gmeanfirst
        f.write(f"\n===================================================================================================================\n")
    f.write(f"\nFuzzy SVM name_method = '{namemethod}',name_function = '{namefunction}'")
    f.write(f"\n*** T = {tmax}; K = {n_neighbors}; GmeanMax = {gmax:0.4f}\n")
    f.write(f"\n*** T = {t_semax}; K = {n_neighbors}; SeMax = {semax:0.4f}\n")
    f.write(f"\n===================================================================================================================\n")
    return weight

In [23]:
import pandas as pd
def compute_average_result(filepath,filename,time,times):
    data = pd.read_csv(filepath)
    sp_svm = se_svm = gm_svm = f1s_svm = acc_svm = auc_svm = 0
    sp_wsvm = se_wsvm = gm_wsvm = f1s_wsvm = acc_wsvm = auc_wsvm = 0
    sp_cen_lin = se_cen_lin = gm_cen_lin = f1s_cen_lin = acc_cen_lin = auc_cen_lin = 0
    sp_cen_exp = se_cen_exp = gm_cen_exp = f1s_cen_exp = acc_cen_exp = auc_cen_exp = 0
    sp_shp_lin = se_shp_lin = gm_shp_lin = f1s_shp_lin = acc_shp_lin = auc_shp_lin = 0
    sp_shp_exp = se_shp_exp = gm_shp_exp = f1s_shp_exp = acc_shp_exp = auc_shp_exp = 0
    sp_hyp_lin = se_hyp_lin = gm_hyp_lin = f1s_hyp_lin = acc_hyp_lin = auc_hyp_lin = 0
    sp_hyp_exp = se_hyp_exp = gm_hyp_exp = f1s_hyp_exp = acc_hyp_exp = auc_hyp_exp = 0
    sp_new1 = se_new1 = gm_new1 = f1s_new1 = acc_new1 = auc_new1 = 0
    sp_new2 = se_new2 = gm_new2 = f1s_new2 = acc_new2 = auc_new2 = 0
    sp_new3 = se_new3 = gm_new3 = f1s_new3 = acc_new3 = auc_new3 = 0

    for i in range(0,len(data)):
        # if(data['Times'][i] == '1'):
        if (data['Name Method'][i] == 'SVM') and (data['Name Function'][i] == 'SVM'):
            sp_svm = sp_svm + float(data['SP'][i])
            se_svm = se_svm + float(data['SE'][i])
            gm_svm = gm_svm + float(data['Gmean'][i])
            f1s_svm = f1s_svm + float(data['F1 Score'][i])
            acc_svm = acc_svm + float(data['Accuracy'][i])
            auc_svm = auc_svm + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'WSVM') and (data['Name Function'][i] == 'WSVM'):
            sp_wsvm = sp_wsvm + float(data['SP'][i])
            se_wsvm = se_wsvm + float(data['SE'][i])
            gm_wsvm = gm_wsvm + float(data['Gmean'][i])
            f1s_wsvm = f1s_wsvm + float(data['F1 Score'][i])
            acc_wsvm = acc_wsvm + float(data['Accuracy'][i])
            auc_wsvm = auc_wsvm + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'own_class_center') and (data['Name Function'][i] == 'lin_center_own'):
            sp_cen_lin = sp_cen_lin + float(data['SP'][i])
            se_cen_lin = se_cen_lin + float(data['SE'][i])
            gm_cen_lin = gm_cen_lin + float(data['Gmean'][i])
            f1s_cen_lin = f1s_cen_lin + float(data['F1 Score'][i])
            acc_cen_lin = acc_cen_lin + float(data['Accuracy'][i])
            auc_cen_lin = auc_cen_lin + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'own_class_center') and (data['Name Function'][i] == 'exp'):
            sp_cen_exp = sp_cen_exp + float(data['SP'][i])
            se_cen_exp = se_cen_exp+ float(data['SE'][i])
            gm_cen_exp = gm_cen_exp + float(data['Gmean'][i])
            f1s_cen_exp = f1s_cen_exp + float(data['F1 Score'][i])
            acc_cen_exp = acc_cen_exp + float(data['Accuracy'][i])
            auc_cen_exp = auc_cen_exp + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'estimated_hyper_lin') and (data['Name Function'][i] == 'lin_center_own'):
            sp_shp_lin = sp_shp_lin + float(data['SP'][i])
            se_shp_lin = se_shp_lin + float(data['SE'][i])
            gm_shp_lin = gm_shp_lin + float(data['Gmean'][i])
            f1s_shp_lin = f1s_shp_lin + float(data['F1 Score'][i])
            acc_shp_lin = acc_shp_lin + float(data['Accuracy'][i])
            auc_shp_lin = auc_shp_lin + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'estimated_hyper_lin') and (data['Name Function'][i] == 'exp'):
            sp_shp_exp = sp_shp_exp + float(data['SP'][i])
            se_shp_exp = se_shp_exp+ float(data['SE'][i])
            gm_shp_exp = gm_shp_exp + float(data['Gmean'][i])
            f1s_shp_exp = f1s_shp_exp + float(data['F1 Score'][i])
            acc_shp_exp = acc_shp_exp + float(data['Accuracy'][i])
            auc_shp_exp = auc_shp_exp + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'actual_hyper_lin') and (data['Name Function'][i] == 'lin_center_own'):
            sp_hyp_lin = sp_hyp_lin + float(data['SP'][i])
            se_hyp_lin = se_hyp_lin + float(data['SE'][i])
            gm_hyp_lin = gm_hyp_lin + float(data['Gmean'][i])
            f1s_hyp_lin = f1s_hyp_lin + float(data['F1 Score'][i])
            acc_hyp_lin = acc_hyp_lin + float(data['Accuracy'][i])
            auc_hyp_lin = auc_hyp_lin + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'actual_hyper_lin') and (data['Name Function'][i] == 'exp'):
            sp_hyp_exp = sp_hyp_exp + float(data['SP'][i])
            se_hyp_exp = se_hyp_exp + float(data['SE'][i])
            gm_hyp_exp = gm_hyp_exp + float(data['Gmean'][i])
            f1s_hyp_exp = f1s_hyp_exp + float(data['F1 Score'][i])
            acc_hyp_exp = acc_hyp_exp + float(data['Accuracy'][i])
            auc_hyp_exp = auc_hyp_exp + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'distance_center_own_opposite_tam') and (data['Name Function'][i] == 'func_own_opp_new'):
            sp_new1 = sp_new1 + float(data['SP'][i])
            se_new1 = se_new1 + float(data['SE'][i])
            gm_new1 = gm_new1 + float(data['Gmean'][i])
            f1s_new1 = f1s_new1 + float(data['F1 Score'][i])
            acc_new1 = acc_new1 + float(data['Accuracy'][i])
            auc_new1 = auc_new1 + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'distance_center_own_opposite_tam') and (data['Name Function'][i] == 'func_own_opp_new_v1'):
            sp_new2 = sp_new2 + float(data['SP'][i])
            se_new2 = se_new2 + float(data['SE'][i])
            gm_new2 = gm_new2 + float(data['Gmean'][i])
            f1s_new2 = f1s_new2 + float(data['F1 Score'][i])
            acc_new2 = acc_new2 + float(data['Accuracy'][i])
            auc_new2 = auc_new2 + float(data['AUC'][i])
        elif (data['Name Method'][i] == 'distance_center_own_opposite_tam') and (data['Name Function'][i] == 'func_own_opp_new_v2'):
            sp_new3= sp_new3 + float(data['SP'][i])
            se_new3 = se_new3 + float(data['SE'][i])
            gm_new3 = gm_new3 + float(data['Gmean'][i])
            f1s_new3 = f1s_new3 + float(data['F1 Score'][i])
            acc_new3 = acc_new3 + float(data['Accuracy'][i])
            auc_new3 = auc_new3 + float(data['AUC'][i])
        else:
            print("end")
    header = ['Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC']
    data = [['SVM','SVM',sp_svm/(5*times),se_svm/(5*times),gm_svm/(5*times),f1s_svm/(5*times),acc_svm/(5*times),auc_svm/(5*times)],
            ['WSVM','WSVM',sp_wsvm/(5*times),se_wsvm/(5*times),gm_wsvm/(5*times),f1s_wsvm/(5*times),acc_wsvm/(5*times),auc_wsvm/(5*times)],
            ['own_class_center','lin_center_own',sp_cen_lin/(5*times),se_cen_lin/(5*times),gm_cen_lin/(5*times),f1s_cen_lin/(5*times),acc_cen_lin/(5*times),auc_cen_lin/(5*times)],
            ['own_class_center','exp',sp_cen_exp/(5*times),se_cen_exp/(5*times),gm_cen_exp/(5*times),f1s_cen_exp/(5*times),acc_cen_exp/(5*times),auc_cen_exp/(5*times)],
            ['estimated_hyper_lin','lin_center_own',sp_shp_lin /(5*times),se_shp_lin/(5*times),gm_shp_lin/(5*times),f1s_shp_lin/(5*times),acc_shp_lin/(5*times),auc_shp_lin /(5*times)],
            ['estimated_hyper_lin','exp',sp_shp_exp/(5*times),se_shp_exp /(5*times),gm_shp_exp/(5*times),f1s_shp_exp/(5*times),acc_shp_exp/(5*times),auc_shp_exp/(5*times)],
            ['actual_hyper_lin','lin_center_own',sp_hyp_lin/(5*times),se_hyp_lin/(5*times),gm_hyp_lin/(5*times),f1s_hyp_lin/(5*times),acc_hyp_lin/(5*times),auc_hyp_lin/(5*times)],
            ['actual_hyper_lin','exp',sp_hyp_exp/(5*times),se_hyp_exp/(5*times),gm_hyp_exp/(5*times),f1s_hyp_exp/(5*times),acc_hyp_exp/(5*times),auc_hyp_exp/(5*times)],
            ['distance_center_own_opposite_tam','func_own_opp_new',sp_new1/(5*times),se_new1/(5*times),gm_new1/(5*times),f1s_new1/(5*times),acc_new1/(5*times),auc_new1/(5*times)],
            ['distance_center_own_opposite_tam','func_own_opp_new_v1',sp_new2/(5*times),se_new2/(5*times),gm_new2/(5*times),f1s_new2/(5*times),acc_new2/(5*times),auc_new2/(5*times)],
            ['distance_center_own_opposite_tam','func_own_opp_new_v2',sp_new3/(5*times),se_new3/(5*times),gm_new3/(5*times),f1s_new3/(5*times),acc_new3/(5*times),auc_new3/(5*times)]]

    with open(f'./Experiment/Data_{filename}_{time}_Average.csv', 'a', encoding='UTF8', newline='') as f4:
        writer = csv.writer(f4)
        writer.writerow(header)
        writer.writerows(data)

In [24]:
name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1", "func_own_opp_new_v2"]
for namemethod in name_method:
    for namefunction in name_function:
        if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
            continue
        elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
            continue
        # elif namemethod == "distance_center_own_opposite_tam" and namefunction == "lin":
        #     continue
        # elif namemethod == "own_class_center" and namefunction == "lin":
        #     continue
        # elif namemethod == "estimated_hyper_lin" and namefunction == "lin":
        #     continue
        # elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
        #    continue
        # elif namemethod == "own_class_center" and namefunction == "lin_center_own":
        #     continue 
        # elif namemethod == "own_class_center" and namefunction == "exp":
        #     continue 
        # elif namemethod == "estimated_hyper_lin" and namefunction == "lin_center_own":
        #     continue 
        # elif namemethod == "estimated_hyper_lin" and namefunction == "exp":
        #     continue 
        # elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
        #     continue
        # elif namemethod == "actual_hyper_lin" and namefunction == "exp":
        #     continue
        else:
            print(namemethod, namefunction)

own_class_center lin_center_own
own_class_center exp
estimated_hyper_lin lin_center_own
estimated_hyper_lin exp
actual_hyper_lin lin_center_own
actual_hyper_lin exp
distance_center_own_opposite_tam func_own_opp_new
distance_center_own_opposite_tam func_own_opp_new_v1
distance_center_own_opposite_tam func_own_opp_new_v2


In [25]:
# C = 100

# thamso1 = 1
# thamso2 = 1
# T = 20
# N = 1
# n_neighbor = 5
# test_size = [0.2]
# testsize_val = 0.2
# K_big = 5
# K_small = 5
# new_rate = [1/3]
# # data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
# #             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]

# #Haberman dataset
# # dataset = Haberman_KFold
# # beta_center, beta_estimate, beta_actual = 1, 1, 0.6 # !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!

# # Ecoli dataset
# # dataset = Ecoli_Kfold
# # beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7
# dataset = Co_Author_TestSize
# #Co-Author dataset

# beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7

# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# #name_method =["own_class_center_divided"]
# name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1"]

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filepath = "./text_script"

# #svc lib
# svc = SVC(probability=True, kernel='linear')
# #svm scratch
# svm_scr = Svm(C)
# #W.svm
# # for n in range(0,N):
# #     print("Lan boc: ",n+1)
#     # for dataset in data:
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# f = open(f"{filepath}/Data_{filename}_{time}_Detail.txt", "w")
# f.write(f"\nC = {C}, thamso1 = {thamso1}, thamso2 = {thamso2}, T = {T}, n_neighbors = {n_neighbor}  \n")
# f.write(f"\n\n\tUSING DATASET : {filename}\n")

# f2 = open(f"{filepath}/Data_{filename}_{time}_Main.txt", "w")
# f2.write(f"\nC = {C}, thamso1 = {thamso1}, thamso2 = {thamso2}, T = {T}, n_neighbors = {n_neighbor}  \n")
# f2.write(f"\n\n\tUSING DATASET : {filename}\n")
    

# print(f"\n\tUSING DATASET : {filename}\n")
# for testsize in test_size:
#     for newrate in new_rate:
#         X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize)
#         # X, y = dataset.load_data()
#         # print(X.shape)
#         # kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
#         header = ['Test Size','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#         data = []
#         with open(f'./Experiment/Data_{filename}_{time}_Main.csv', 'a', encoding='UTF8', newline='') as f3:
#             writer = csv.writer(f3)
#             writer.writerow(header)
#             # fold = 1
#             # for train_index, test_index in kfold_validation.split(X,y):
                
#                 # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#                 # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#                 # print(X_test.shape)
#                 # #Scalling Data
#                 # sc_X = StandardScaler()
#                 # X_train = sc_X.fit_transform(X_train)
#                 # X_test = sc_X.transform(X_test)
#                 # y_train = np.array(y_train)

#                 # print(type(X_train), type(y_train))

#                 #print(f"\t======== TestSize: {testsize} ========")
#                 #print("So luong sample nguyen ban ban dau: ",len(X_train)+len(X_test_val)+len(X_test))

#                 # f.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
#             f.write("\n\n\t====== NOT USING TOMEKLINKS ========== \n")

#             # f2.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
#             f2.write("\n\n\t====== NOT USING TOMEKLINKS ========== \n")

#             #Svm library
#             f.write("\n\n\tSVM LIBRARY starting...\n")
#             f2.write("\n\n\tSVM LIBRARY starting...\n")
#             #print("SVM LIBRARY starting...\n")
#             test_pred = svm_lib(X_train, y_train,X_test)
#             sp,se,gmean = Gmean(y_test,test_pred)
#             #metr(X_train,y_test,test_pred,sp,se,gmean)
#             metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
#             metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)
#             name1 = 'SVM'
#             name2 = 'SVM'
#             data.append([testsize,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            
#             # #Svm scratch
#             # f.write("\n\n\tSVM starting...\n")
#             # f2.write("\n\n\tSVM starting...\n")
#             # print("SVM starting...\n")
#             # test_pred = svm(C,X_train, y_train,X_test)
#             # sp,se,gmean = Gmean(y_test,test_pred)
#             # #metr(X_train,y_test,test_pred,sp,se,gmean)
#             # metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
#             # metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)

#             #Wsvm
#             f.write("\n\n\tW.SVM starting...\n")
#             f2.write("\n\n\tW.SVM starting...\n")
#             print("W.SVM starting...\n")
#             N, d = X_train.shape
#             distribution_weight = np.ones(N)

#             #header = ['Fold','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#             #data1 = []
#             #with open('Result_C.csv', 'w', encoding='UTF8', newline='') as f4:
#                 ##writer1.writerow(header)
#             # for i in range(1,15):
#             #     C = 2**i
#             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#             sp,se,gmean = Gmean(y_test,test_pred)
#             #     #metr(X_train,y_test,test_pred,sp,se,gmean)
#             metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
#             metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)
#             name1 = 'WSVM'
#             name2 = 'WSVM'
#             #     data.append([fold,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#             data.append([testsize,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#             # writer.writerows(data)

#             #FuzyyWsvm
#             for namemethod in name_method:
#                 for namefunction in name_function:
#                     if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                         continue
#                     elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     else:
#                         f.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                         f2.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                         print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                         distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                         __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                         test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                         sp,se,gmean = Gmean(y_test,test_pred)
#                         metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
#                         metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)
#                         data.append([testsize,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#             # # f.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
#             # f.write("\n\n\t====== USING TOMEKLINKS ========== \n")

#             # # f2.write(f"\n\n\t======== TestSize: {testsize} ========\n\n")
#             # f2.write("\n\n\t====== USING TOMEKLINKS ========== \n")

#             # arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1])
#             # #FuzyyWsvm
#             # for namemethod in name_method:
#             #     for namefunction in name_function:
#             #         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#             #             continue
#             #         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#             #             continue
#             #         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#             #             continue
#             #         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#             #             continue
#             #         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#             #             continue
#             #         # elif namemethod == "distance_center_own_opposite_tam" and namefunction == "lin":
#             #         #     continue
#             #         # elif namemethod == "own_class_center" and namefunction == "lin":
#             #         #     continue
#             #         # elif namemethod == "estimated_hyper_lin" and namefunction == "lin":
#             #         #     continue
#             #         # elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
#             #         # #     continue
#             #         # elif namemethod == "own_class_center" and namefunction == "lin_center_own":
#             #         #     continue 
#             #         # elif namemethod == "own_class_center" and namefunction == "exp":
#             #         #     continue 
#             #         # elif namemethod == "estimated_hyper_lin" and namefunction == "lin_center_own":
#             #         #     continue 
#             #         # elif namemethod == "estimated_hyper_lin" and namefunction == "exp":
#             #         #     continue 
#             #         # elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
#             #         #     continue
#             #         # elif namemethod == "actual_hyper_lin" and namefunction == "exp":
#             #         #     continue
#             #         else:
#             #             f.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#             #             f2.write(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#             #             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#             #             distribution_weight = fuzzy_weight(f,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#             #             __ = fuzzy_weight(f2,beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#             #             # fold_a = f"{fold}_a"
                        
#             #             for i in range(0,T):
#             #                 # new_W = lfb(f,C,distribution_weight,namemethod,namefunction,T,X_test,y_test,X_train,y_train,n_neighbor,thamso1,thamso2)
#             #                 new_W = data_tomelinks_new1(f,C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
#             #                 test_pred = wsvm(C,X_train, y_train, X_test, new_W)
#             #                 sp,se,gmean = Gmean(y_test,test_pred)
#             #                 #metr(X_train,y_test,test_pred,sp,se,gmean)
#             #                 metr_text(f,X_train,y_test,test_pred,sp,se,gmean)
#             #                 metr_text(f2,X_train,y_test,test_pred,sp,se,gmean)
#             #                 # fold_as=f"{fold_a}_{i+1}"
#             #                 data.append([testsize,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#             # # fold = fold + 1
#             writer.writerows(data)
#         f.write("\n===================================================================================\n")
#         f.close()
#         f2.write("\n===================================================================================\n")
#         f2.close()

In [26]:
# C = 100
# thamso1 = 1
# thamso2 = 1
# T = 5
# N = 5
# n_neighbor = 5
# test_size = [0.2,0.3,0.4]
# testsize_val = 0.2
# K_big = 5
# K_small = 5
# new_rate = [1/5, 1/7, 1/9]
# # data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
# #             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]

# #Haberman dataset
# # dataset = Haberman_KFold
# # beta_center, beta_estimate, beta_actual = 1, 1, 0.6 # !!!!!!! Beta with Dataset, change Data please change Beta !!!!!!!!

# # Ecoli dataset
# # dataset = Ecoli_Kfold
# # beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7

# # dataset = Pima
# # beta_center, beta_estimate, beta_actual = 0.5, 1, 0.5

# # dataset = Transfution_Kfold
# # beta_center, beta_estimate, beta_actual = 0.5, 0.8, 0.1
# from Processing_Data import Co_Author_50_250
# dataset = Co_Author_50_250
# beta_center, beta_estimate, beta_actual = 0.5, 0.8, 0.1

# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_Full.csv'

# #svc lib
# svc = SVC(probability=True, kernel='linear')
# #svm scratch
# svm_scr = Svm(C)
# #W.svm
# for n in range(0,N):
#     print("Lan boc: ",n+1)
#     for ir in new_rate:
#         X, y = dataset.load_data(ir)
#         print(X.shape)
#         kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
#         header = ['Times','IR','Fold','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#         data = []
#         with open(f'./Experiment/Data_{filename}_{time}_Full.csv', 'a', encoding='UTF8', newline='') as f3:
#             writer = csv.writer(f3)
#             writer.writerow(header)
#             fold = 1
#             for train_index, test_index in kfold_validation.split(X,y):
                
#                 # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#                 # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#                 X_train, y_train = X[train_index], y[train_index]
#                 X_test, y_test = X[test_index], y[test_index]
#                 print(X_test.shape)
                
#                 #Scalling Data
#                 sc_X = StandardScaler()
#                 X_train = sc_X.fit_transform(X_train)
#                 X_test = sc_X.transform(X_test)
#                 y_train = np.array(y_train)

#                 # NOT USING TOMEK LINKS

#                 #Svm library
#                 print("SVM LIBRARY starting...\n")
#                 test_pred = svm_lib(X_train, y_train,X_test)
#                 sp,se,gmean = Gmean(y_test,test_pred)
#                 name1 = 'SVM'
#                 name2 = 'SVM'
#                 data.append([n,ir,fold,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                
#                 #Wsvm
#                 print("W.SVM starting...\n")
#                 N, d = X_train.shape
#                 distribution_weight = np.ones(N)
#                 test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                 sp,se,gmean = Gmean(y_test,test_pred)
#                 name1 = 'WSVM'
#                 name2 = 'WSVM'
#                 data.append([n,ir,fold,name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#                 #FuzyyWsvm
#                 for namemethod in name_method:
#                     for namefunction in name_function:
#                         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                             continue
#                         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         else:
#                             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                             distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                             __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                             sp,se,gmean = Gmean(y_test,test_pred)
#                             data.append([n,ir,fold,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#                 fold = fold + 1
#             writer.writerows(data)
# # compute_average_result(filepath,filename,time,N)

In [ ]:
################################# K-FOLD SCRIPT FOR CO-AUTHOR DATASETS ##############################
C = 100
T = 1
N = 3
n_neighbor = 5
test_size = [0.2,0.3,0.4]
K_big = 5
K_small = 5
new_rate = [1/5, 1/7, 1/9]

from Processing_Data import Co_Author_50_250, Co_Author_250_750, Co_Author_100_900, Co_Author_100_700, Co_Author_100_500, Co_Author_50_350, Co_Author_200_1000, Co_Author_200_1000_1, Co_Author_300_1500
dataset = Co_Author_200_1000
beta_center, beta_estimate, beta_actual = 0.5, 0.8, 0.1

name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]

time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]

#W.svm
for n in range(0,N):
    print("Lan boc: ",n+1)
    X, y = dataset.load_data()
    print(X.shape)
    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
    header = ['Times','Fold','T','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
    data = []
    with open(f'./Experiment/KFold_Data_{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
        writer = csv.writer(f3)
        writer.writerow(header)
        fold = 1
        for train_index, test_index in kfold_validation.split(X,y):
            
            # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
            # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            print(X_test.shape)
            
            #Scalling Data
            sc_X = StandardScaler()
            X_train = sc_X.fit_transform(X_train)
            X_test = sc_X.transform(X_test)
            y_train = np.array(y_train)

            # NORMAL

            #Svm library
            print("SVM LIBRARY starting...\n")
            test_pred = svm_lib(X_train, y_train,X_test)
            sp,se,gmean = Gmean(y_test,test_pred)
            name1 = 'SVM'
            name2 = 'SVM'
            data.append([n,fold,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            
            #Wsvm
            print("W.SVM starting...\n")
            N, d = X_train.shape
            distribution_weight = np.ones(N)
            test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
            sp,se,gmean = Gmean(y_test,test_pred)
            name1 = 'WSVM'
            name2 = 'WSVM'
            data.append([n,fold,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

            #FuzyyWsvm
            for namemethod in name_method:
                for namefunction in name_function:
                    if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                        continue
                    elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    else:
                        print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                        distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
                        sp,se,gmean = Gmean(y_test,test_pred)
                        data.append([n,fold,"None",namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

            ############ USING AFW-CIL METHOD #########################

            arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
            #FuzyyWsvm
            for namemethod in name_method:
                for namefunction in name_function:
                    if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                        continue
                    elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    else:
                        print("===========USING AFW-CIL=====================")
                        print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                        distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        
                        for i in range(0,T):
                            new_W = data_tomelinks_new1(C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
                            test_pred = wsvm(C,X_train, y_train, X_test, new_W)
                            sp,se,gmean = Gmean(y_test,test_pred)
                            data.append([n,fold,i+1,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            fold = fold + 1
        writer.writerows(data)
    
# compute_average_result(filepath,filename,time,N)

In [ ]:
###################### TEST-SIZE SCRIPT FOR CO-AUTHOR DATASETS #################################
C = 100
T = 20
N = 1
n_neighbor = 5
test_size = [0.2]
new_rate = [1/3]

dataset = Ecoli_All
beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7

name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]
time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]


for n in range(0,N):
    print("Lần bốc thứ: ",n+1)
    header = ['Lan boc','Test Size','Rate','T','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
    data = []
    for testsize in test_size:
        for newrate in new_rate:
            X_train, y_train, X_test, y_test = dataset.load_data()
            with open(f'./Experiment/TestSize_Data_{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
                writer = csv.writer(f3)
                writer.writerow(header)
                
                #Scalling Data
                sc_X = StandardScaler()
                X_train = sc_X.fit_transform(X_train)
                X_test = sc_X.transform(X_test)
                y_train = np.array(y_train)

                # NORMAL

                #Svm library
                print("SVM LIBRARY starting...\n")
                test_pred = svm_lib(X_train, y_train,X_test)
                sp,se,gmean = Gmean(y_test,test_pred)
                name1 = 'SVM'
                name2 = 'SVM'
                data.append([n+1,testsize,newrate,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                
                #Wsvm
                print("W.SVM starting...\n")
                N, d = X_train.shape
                distribution_weight = np.ones(N)
                test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
                sp,se,gmean = Gmean(y_test,test_pred)
                name1 = 'WSVM'
                name2 = 'WSVM'
                data.append([n+1,testsize,newrate,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

                #FuzyyWsvm
                for namemethod in name_method:
                    for namefunction in name_function:
                        if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                            continue
                        elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                            continue
                        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                            continue
                        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                            continue
                        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                            continue
                        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
                            continue
                        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
                            continue
                        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
                            continue
                        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                            continue
                        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                            continue
                        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                            continue
                        else:
                            print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                            distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                            __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                            test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
                            sp,se,gmean = Gmean(y_test,test_pred)
                            data.append([n+1,testsize,newrate,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

                ############ USING AFW-CIL METHOD #########################
                arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
                #FuzyyWsvm
                for namemethod in name_method:
                    for namefunction in name_function:
                        if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                            continue
                        elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                            continue
                        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                            continue
                        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                            continue
                        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                            continue
                        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
                            continue
                        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
                            continue
                        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
                            continue
                        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                            continue
                        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                            continue
                        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                            continue
                        else:
                            print("===========USING AFW-CIL=====================")
                            print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                            distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                            __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                            
                            for i in range(0,T):
                                new_W = data_tomelinks_new1(C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
                                test_pred = wsvm(C,X_train, y_train, X_test, new_W)
                                sp,se,gmean = Gmean(y_test,test_pred)
                                data.append([n+1,testsize,newrate,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                writer.writerows(data)


In [ ]:
f3.close()

In [ ]:
# import pandas as pd
# from datetime import datetime
# import csv
# filepath = "D:/MULTIMEDIA/MACHINE_LEARNING_THAY_QUANG/FUZZY SVM/CODE/07_04_2022/fuzzy_svm/Experiment/Data_Co_Author_200_1000_28092022_093028_Full.csv"
# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = "Co_Author_200_1000"
# def compute_average_result(filepath,filename,time):
#     data = pd.read_csv(filepath)
#     sp_svm = se_svm = gm_svm = f1s_svm = acc_svm = auc_svm = 0
#     sp_wsvm = se_wsvm = gm_wsvm = f1s_wsvm = acc_wsvm = auc_wsvm = 0
#     sp_cen_lin = se_cen_lin = gm_cen_lin = f1s_cen_lin = acc_cen_lin = auc_cen_lin = 0
#     sp_cen_exp = se_cen_exp = gm_cen_exp = f1s_cen_exp = acc_cen_exp = auc_cen_exp = 0
#     sp_shp_lin = se_shp_lin = gm_shp_lin = f1s_shp_lin = acc_shp_lin = auc_shp_lin = 0
#     sp_shp_exp = se_shp_exp = gm_shp_exp = f1s_shp_exp = acc_shp_exp = auc_shp_exp = 0
#     sp_hyp_lin = se_hyp_lin = gm_hyp_lin = f1s_hyp_lin = acc_hyp_lin = auc_hyp_lin = 0
#     sp_hyp_exp = se_hyp_exp = gm_hyp_exp = f1s_hyp_exp = acc_hyp_exp = auc_hyp_exp = 0
#     sp_new1 = se_new1 = gm_new1 = f1s_new1 = acc_new1 = auc_new1 = 0
#     sp_new2 = se_new2 = gm_new2 = f1s_new2 = acc_new2 = auc_new2 = 0
#     sp_new3 = se_new3 = gm_new3 = f1s_new3 = acc_new3 = auc_new3 = 0

#     for i in range(0,len(data)):
#         # if(data['Times'][i] == '1'):
#         if (data['Name Method'][i] == 'SVM') and (data['Name Function'][i] == 'SVM'):
#             sp_svm = sp_svm + float(data['SP'][i])
#             se_svm = se_svm + float(data['SE'][i])
#             gm_svm = gm_svm + float(data['Gmean'][i])
#             f1s_svm = f1s_svm + float(data['F1 Score'][i])
#             acc_svm = acc_svm + float(data['Accuracy'][i])
#             auc_svm = auc_svm + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'WSVM') and (data['Name Function'][i] == 'WSVM'):
#             sp_wsvm = sp_wsvm + float(data['SP'][i])
#             se_wsvm = se_wsvm + float(data['SE'][i])
#             gm_wsvm = gm_wsvm + float(data['Gmean'][i])
#             f1s_wsvm = f1s_wsvm + float(data['F1 Score'][i])
#             acc_wsvm = acc_wsvm + float(data['Accuracy'][i])
#             auc_wsvm = auc_wsvm + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'own_class_center') and (data['Name Function'][i] == 'lin_center_own'):
#             sp_cen_lin = sp_cen_lin + float(data['SP'][i])
#             se_cen_lin = se_cen_lin + float(data['SE'][i])
#             gm_cen_lin = gm_cen_lin + float(data['Gmean'][i])
#             f1s_cen_lin = f1s_cen_lin + float(data['F1 Score'][i])
#             acc_cen_lin = acc_cen_lin + float(data['Accuracy'][i])
#             auc_cen_lin = auc_cen_lin + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'own_class_center') and (data['Name Function'][i] == 'exp'):
#             sp_cen_exp = sp_cen_exp + float(data['SP'][i])
#             se_cen_exp = se_cen_exp+ float(data['SE'][i])
#             gm_cen_exp = gm_cen_exp + float(data['Gmean'][i])
#             f1s_cen_exp = f1s_cen_exp + float(data['F1 Score'][i])
#             acc_cen_exp = acc_cen_exp + float(data['Accuracy'][i])
#             auc_cen_exp = auc_cen_exp + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'estimated_hyper_lin') and (data['Name Function'][i] == 'lin_center_own'):
#             sp_shp_lin = sp_shp_lin + float(data['SP'][i])
#             se_shp_lin = se_shp_lin + float(data['SE'][i])
#             gm_shp_lin = gm_shp_lin + float(data['Gmean'][i])
#             f1s_shp_lin = f1s_shp_lin + float(data['F1 Score'][i])
#             acc_shp_lin = acc_shp_lin + float(data['Accuracy'][i])
#             auc_shp_lin = auc_shp_lin + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'estimated_hyper_lin') and (data['Name Function'][i] == 'exp'):
#             sp_shp_exp = sp_shp_exp + float(data['SP'][i])
#             se_shp_exp = se_shp_exp+ float(data['SE'][i])
#             gm_shp_exp = gm_shp_exp + float(data['Gmean'][i])
#             f1s_shp_exp = f1s_shp_exp + float(data['F1 Score'][i])
#             acc_shp_exp = acc_shp_exp + float(data['Accuracy'][i])
#             auc_shp_exp = auc_shp_exp + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'actual_hyper_lin') and (data['Name Function'][i] == 'lin_center_own'):
#             sp_hyp_lin = sp_hyp_lin + float(data['SP'][i])
#             se_hyp_lin = se_hyp_lin + float(data['SE'][i])
#             gm_hyp_lin = gm_hyp_lin + float(data['Gmean'][i])
#             f1s_hyp_lin = f1s_hyp_lin + float(data['F1 Score'][i])
#             acc_hyp_lin = acc_hyp_lin + float(data['Accuracy'][i])
#             auc_hyp_lin = auc_hyp_lin + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'actual_hyper_lin') and (data['Name Function'][i] == 'exp'):
#             sp_hyp_exp = sp_hyp_exp + float(data['SP'][i])
#             se_hyp_exp = se_hyp_exp + float(data['SE'][i])
#             gm_hyp_exp = gm_hyp_exp + float(data['Gmean'][i])
#             f1s_hyp_exp = f1s_hyp_exp + float(data['F1 Score'][i])
#             acc_hyp_exp = acc_hyp_exp + float(data['Accuracy'][i])
#             auc_hyp_exp = auc_hyp_exp + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'distance_center_own_opposite_tam') and (data['Name Function'][i] == 'func_own_opp_new'):
#             sp_new1 = sp_new1 + float(data['SP'][i])
#             se_new1 = se_new1 + float(data['SE'][i])
#             gm_new1 = gm_new1 + float(data['Gmean'][i])
#             f1s_new1 = f1s_new1 + float(data['F1 Score'][i])
#             acc_new1 = acc_new1 + float(data['Accuracy'][i])
#             auc_new1 = auc_new1 + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'distance_center_own_opposite_tam') and (data['Name Function'][i] == 'func_own_opp_new_v1'):
#             sp_new2 = sp_new2 + float(data['SP'][i])
#             se_new2 = se_new2 + float(data['SE'][i])
#             gm_new2 = gm_new2 + float(data['Gmean'][i])
#             f1s_new2 = f1s_new2 + float(data['F1 Score'][i])
#             acc_new2 = acc_new2 + float(data['Accuracy'][i])
#             auc_new2 = auc_new2 + float(data['AUC'][i])
#         elif (data['Name Method'][i] == 'distance_center_own_opposite_tam') and (data['Name Function'][i] == 'func_own_opp_new_v2'):
#             sp_new3= sp_new3 + float(data['SP'][i])
#             se_new3 = se_new3 + float(data['SE'][i])
#             gm_new3 = gm_new3 + float(data['Gmean'][i])
#             f1s_new3 = f1s_new3 + float(data['F1 Score'][i])
#             acc_new3 = acc_new3 + float(data['Accuracy'][i])
#             auc_new3 = auc_new3 + float(data['AUC'][i])
#         else:
#             print("end")
#     a = 15
#     header = ['Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC']
#     data = [['SVM','SVM',sp_svm/a,se_svm/a,gm_svm/a,f1s_svm/a,acc_svm/a,auc_svm/a],
#             ['WSVM','WSVM',sp_wsvm/a,se_wsvm/a,gm_wsvm/a,f1s_wsvm/a,acc_wsvm/a,auc_wsvm/a],
#             ['own_class_center','lin_center_own',sp_cen_lin/a,se_cen_lin/a,gm_cen_lin/a,f1s_cen_lin/a,acc_cen_lin/a,auc_cen_lin/a],
#             ['own_class_center','exp',sp_cen_exp/a,se_cen_exp/a,gm_cen_exp/a,f1s_cen_exp/a,acc_cen_exp/a,auc_cen_exp/a],
#             ['estimated_hyper_lin','lin_center_own',sp_shp_lin /a,se_shp_lin /a,gm_shp_lin /a,f1s_shp_lin /a,acc_shp_lin /a,auc_shp_lin /a],
#             ['estimated_hyper_lin','exp',sp_shp_exp /a,se_shp_exp /a,gm_shp_exp/a,f1s_shp_exp/a,acc_shp_exp/a,auc_shp_exp/a],
#             ['actual_hyper_lin','lin_center_own',sp_hyp_lin/a,se_hyp_lin/a,gm_hyp_lin/a,f1s_hyp_lin/a,acc_hyp_lin/a,auc_hyp_lin/a],
#             ['actual_hyper_lin','exp',sp_hyp_exp/a,se_hyp_exp/a,gm_hyp_exp/a,f1s_hyp_exp/a,acc_hyp_exp/a,auc_hyp_exp/a],
#             ['distance_center_own_opposite_tam','func_own_opp_new',sp_new1/a,se_new1/a,gm_new1/a,f1s_new1/a,acc_new1/a,auc_new1/a],
#             ['distance_center_own_opposite_tam','func_own_opp_new_v1',sp_new2/a,se_new2/a,gm_new2/a,f1s_new2/a,acc_new2/a,auc_new2/a],
#             ['distance_center_own_opposite_tam','func_own_opp_new_v2',sp_new3/a,se_new3/a,gm_new3/a,f1s_new3/a,acc_new3/a,auc_new3/a]]

#     with open(f'./Experiment/Data_{filename}_{time}_Average.csv', 'a', encoding='UTF8', newline='') as f4:
#         writer = csv.writer(f4)
#         writer.writerow(header)
#         writer.writerows(data)

# compute_average_result(filepath, filename, time)




